# PIMALUOS Pareto Front Visualization

This notebook demonstrates Pareto optimization and visualizes the
multi-objective trade-offs.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 10)

## Run Pareto Optimization

Note: This requires pymoo to be installed.

In [ ]:
# Import PIMALUOS
from pimaluos import UrbanOptSystem
from pimaluos.models import optimize_land_use_pareto

# Initialize system with small subset
print("Initializing system...")
system = UrbanOptSystem(
    data_subset_size=100,
    llm_mode='mock',
    device='cpu'
)

# Load data
print("Loading data...")
system.load_data()
system.build_graph()
system.extract_constraints()
system.initialize_physics_engine()

print(f"✓ Loaded {len(system.gdf)} parcels")

In [ ]:
# Run Pareto optimization
print("Running Pareto optimization (this may take a few minutes)...")

solutions, knee = optimize_land_use_pareto(
    num_parcels=len(system.gdf),
    constraint_masks=system.constraint_masks,
    physics_engine=system.physics_engine,
    gdf=system.gdf,
    algorithm='nsga2',
    population_size=50,  # Small for demo
    num_generations=10   # Small for demo
)

print(f"\n✓ Found {len(solutions)} Pareto-optimal solutions")
print(f"\nKnee solution objectives:")
print(f"  Economic: {knee.objectives[0]:.3f}")
print(f"  Environmental: {knee.objectives[1]:.3f}")
print(f"  Social: {knee.objectives[2]:.3f}")
print(f"  Equity: {knee.objectives[3]:.3f}")

## Visualize Pareto Front

We'll create 2D projections of the 4D Pareto front.

In [ ]:
# Extract objectives
objectives = np.array([s.objectives for s in solutions])
obj_names = ['Economic', 'Environmental', 'Social', 'Equity']

# Create pairwise scatter plots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

pairs = [
    (0, 1, 'Economic vs Environmental'),
    (0, 2, 'Economic vs Social'),
    (0, 3, 'Economic vs Equity'),
    (1, 2, 'Environmental vs Social'),
    (1, 3, 'Environmental vs Equity'),
    (2, 3, 'Social vs Equity')
]

for idx, (i, j, title) in enumerate(pairs):
    ax = axes[idx]
    
    # Plot all solutions
    ax.scatter(objectives[:, i], objectives[:, j], 
               alpha=0.6, s=50, c='steelblue', label='Pareto solutions')
    
    # Highlight knee solution
    ax.scatter(knee.objectives[i], knee.objectives[j],
               s=200, c='red', marker='*', 
               edgecolors='darkred', linewidths=2,
               label='Knee solution', zorder=10)
    
    ax.set_xlabel(obj_names[i], fontsize=12)
    ax.set_ylabel(obj_names[j], fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/pareto_front_2d.png', dpi=300, bbox_inches='tight')
plt.show()

## 3D Pareto Front Visualization

In [ ]:
# 3D plot: Economic, Environmental, Social (color by Equity)
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(objectives[:, 0], objectives[:, 1], objectives[:, 2],
                    c=objectives[:, 3], cmap='viridis', 
                    s=50, alpha=0.6)

# Highlight knee solution
ax.scatter(knee.objectives[0], knee.objectives[1], knee.objectives[2],
          s=300, c='red', marker='*', 
          edgecolors='darkred', linewidths=2)

ax.set_xlabel('Economic', fontsize=12)
ax.set_ylabel('Environmental', fontsize=12)
ax.set_zlabel('Social', fontsize=12)
ax.set_title('3D Pareto Front\n(Color = Equity)', 
             fontsize=14, fontweight='bold')

cbar = plt.colorbar(scatter, ax=ax, shrink=0.5, aspect=5)
cbar.set_label('Equity', fontsize=12)

plt.savefig('../results/pareto_front_3d.png', dpi=300, bbox_inches='tight')
plt.show()

## Trade-off Analysis

In [ ]:
# Compute correlations between objectives
corr = pd.DataFrame(objectives, columns=obj_names).corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={'shrink': 0.8})
plt.title('Objective Correlations', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../results/objective_correlations.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nStrong trade-offs (|correlation| > 0.5):")
for i in range(4):
    for j in range(i+1, 4):
        if abs(corr.iloc[i, j]) > 0.5:
            relationship = "negative" if corr.iloc[i,j] < 0 else "positive"
            print(f"  {obj_names[i]} vs {obj_names[j]}: {corr.iloc[i,j]:.3f} ({relationship})")